# Cycle Manager Tutorial
This tutorial will explain how to use the cycle manager.

In [1]:
import time

from psipy.rl.cycle_manager import CycleManager

## What does the Cycle Manager do?
The cycle manager is an object which contains timers, each of which records cycle times of various processes.  The timers calculate an exponential weighted moving average (EMA) with a period of 100 centiseconds.  In other words, it is the weighted average over the past second.

The cycle manager also deals with ZMQ communication between the loop and the TUI.  It also can hold diagnostic information to be sent to a third party.  Essentially, it is a workhorse which can do many things related to managing the cycle.

## How to use the Cycle Manager $-$ For Timing
Instantiate the manager first.  To start a timer, just use the manager as a dictionary and "open" a new key with the ```.tick()``` method, e.g. ```cycle_manager['Tutorial'].tick()```.  Once what you are trying to record is over, call the ```.tock()``` method.  Since this may make the code very cluttered, you can also use ```with```, in which case you do not need to call ```.tick()```.  See the example below:

In [2]:
def loop(manager):
    manager['task1'].tick()
    time.sleep(.5)
    manager['task1'].tock()
    # This is equivalent to:
    #with manager['task1']:
    #    time.sleep(.5)
    
manager = CycleManager()

for _ in range(2):
    loop(manager)
    
print("Time (sec, std sec):", manager['task1'].time)

Time (sec, std sec): 0.5051514677482076


## Timer Specifics
Timers have a ```time``` property, which automatically accessed the callback data directly.

## Interpreting the EMA
The EMA is weighted such that the latest observations are weighted more.  Below is an example of how such an average behaves.  The loop cycles between .5 and .25 seconds, and so the average should be between those two numbers.  However, since the EMA has a weighting function, the EMA won't equal .375 (the simple average) until it is "full", i.e. it has seen 100 observations.  

Note, this takes a couple of minutes to run!

In [3]:
def loop(manager):
    with manager['task1']:
        time.sleep(.5)
    with manager['task1']:
        time.sleep(.25)
    
manager = CycleManager()

for _ in range(200):
    loop(manager)
    
print("Time (sec, std sec):", manager['task1'].time)

Time (sec, std sec): 0.3781172094899834


Note that the average time is around .375, which is equivalent to:

In [ ]:
(.5+.25)/2

which is the simple average time.